# FIFA World Cup 2026 — Simulated with bracketeer

The 2026 FIFA World Cup introduces an expanded 48-team format: 12 groups of 4,
72 group-stage matches, and a 32-team knockout that plays all the way through
to the final and third-place play-off.

This notebook builds the full competition with bracketeer and simulates it
from kick-off to champion — 104 matches in total.

In [172]:
if (!requireNamespace("bracketeer", quietly = TRUE)) {
  install.packages("devtools")
  devtools::install_github("bbtheo/bracketeer")
}
library(bracketeer)

## The format

| Phase | Details | Matches |
|---|---|---|
| Group stage | 12 groups of 4, single round-robin | 72 |
| Round of 32 | Top 2 per group + best 8 third-placed teams | 16 |
| Round of 16 | | 8 |
| Quarter-finals | | 4 |
| Semi-finals | | 2 |
| Third-place play-off | | 1 |
| Final | | 1 |
| **Total** | | **104** |

24 teams qualify automatically (top 2 from each of the 12 groups). The
remaining 8 knockout spots go to the best third-placed teams ranked across
all groups by points, then goal difference.

In [173]:
teams <- c(
  "Canada", "Mexico", "United States",
  "Argentina", "Brazil", "Uruguay", "Colombia", "Ecuador",
  "Paraguay", "Chile",
  "England", "France", "Germany", "Spain", "Portugal",
  "Netherlands", "Belgium", "Croatia", "Italy", "Denmark",
  "Switzerland", "Austria", "Serbia", "Poland", "Ukraine",
  "Japan", "Korea Republic", "IR Iran", "Saudi Arabia",
  "Australia", "Qatar", "Uzbekistan", "Jordan",
  "Morocco", "Senegal", "Tunisia", "Algeria", "Egypt",
  "Cote d'Ivoire", "Ghana", "Cameroon", "Nigeria", "South Africa",
  "New Zealand",
  "Costa Rica", "Panama", "Jamaica", "Honduras"
)

cat(length(teams), "teams registered\n")

48 teams registered


## Qualification selector

bracketeer's `filter_by()` accepts a custom function that receives the source
stage's standings and returns the participants to advance. Here we implement
the official qualification rule: top 2 from each group advance automatically,
then the 8 best third-placed finishers are ranked across all 12 groups.

In [174]:
best_32_selector <- filter_by(function(standings = NULL, source_pool = NULL, ...) {
  if (is.null(standings) || !is.data.frame(standings) ||
      !all(c("participant", "group") %in% names(standings))) {
    pool <- as.character(source_pool)
    return(head(pool, min(length(pool), 32L)))
  }

  tbl <- standings
  tbl$participant <- as.character(tbl$participant)
  tbl$group      <- as.character(tbl$group)

  if (!"group_rank" %in% names(tbl)) {
    tbl <- tbl[order(tbl$group, tbl$rank), , drop = FALSE]
    tbl$group_rank <- as.integer(ave(
      tbl$rank, tbl$group,
      FUN = function(x) rank(x, ties.method = "first")
    ))
  }
  if (!"points" %in% names(tbl))     tbl$points     <- 0L
  if (!"score_diff" %in% names(tbl)) tbl$score_diff <- 0L

  auto_qualifiers <- tbl[tbl$group_rank <= 2L, , drop = FALSE]
  thirds          <- tbl[tbl$group_rank == 3L, , drop = FALSE]
  thirds          <- thirds[order(-thirds$points, -thirds$score_diff), , drop = FALSE]
  best_thirds     <- utils::head(thirds, 8L)

  unique(c(as.character(auto_qualifiers$participant),
           as.character(best_thirds$participant)))
})

## Building the tournament

`spec()` defines the structure without committing to a specific field of teams.
`validate()` checks that the routing is feasible before we materialise anything.
`build()` creates the live tournament runtime.

In [175]:
wc2026 <- spec() |>
  round_robin(
    "groups",
    groups       = 12,
    tiebreakers  = c("points", "score_diff", "sos", "head_to_head", "alphabetical")
  ) |>
  single_elim("knockout", third_place = TRUE, take = best_32_selector)

validate(wc2026, n = length(teams))

trn <- build(wc2026, teams)
stage_status(trn)

     stage      status complete total materialized
1   groups in_progress        0    72         TRUE
2 knockout     blocked        0     0        FALSE

## Group stage

72 matches across 12 groups. Every team plays 3 matches. We simulate each
result with a random score and then manually advance — `auto_advance = FALSE`
here so we can inspect the group standings before the knockout draw.

In [176]:
random_score <- function(max_goals = 4L, allow_draw = TRUE) {
  s <- sample.int(max_goals + 1L, 2L, replace = TRUE) - 1L
  if (!allow_draw && s[1L] == s[2L]) {
    i    <- sample.int(2L, 1L)
    s[i] <- s[i] + 1L
  }
  as.numeric(s)
}

group_m <- matches(trn, "groups", status = "all")
cat("Playing", nrow(group_m), "group-stage matches ...\n")

for (i in seq_len(nrow(group_m))) {
  trn <- result(trn, "groups",
    match = group_m$match_id[[i]],
    score = random_score(allow_draw = TRUE),
    auto_advance = FALSE
  )
}

trn <- advance(trn, "groups")
cat("Group stage complete.\n")
stage_status(trn)

Playing 72 group-stage matches ...
Group stage complete.


     stage      status complete total materialized
1   groups    complete       72    72         TRUE
2 knockout in_progress        0    32         TRUE

## Group standings

Final standings for all 12 groups. The top 2 from each group qualify
automatically; 8 additional spots go to the best third-placed teams.

In [177]:
grp <- standings(trn, "groups")

for (g in sort(unique(grp$group))) {
  rows <- grp[grp$group == g, ]
  rows <- rows[order(rows$rank), ]
  cat(sprintf(
    "Group %s  |  %s  %s  %s  %s\n", g,
    rows$participant[1L], rows$participant[2L],
    rows$participant[3L], rows$participant[4L]
  ))
}

Group A  |  Honduras  Canada  Ukraine  Poland
Group B  |  Mexico  Serbia  Jamaica  Japan
Group C  |  United States  Austria  Panama  Korea Republic
Group D  |  Argentina  Switzerland  Costa Rica  IR Iran
Group E  |  Brazil  Denmark  Saudi Arabia  New Zealand
Group F  |  South Africa  Australia  Italy  Uruguay
Group G  |  Qatar  Croatia  Nigeria  Colombia
Group H  |  Ecuador  Cameroon  Belgium  Uzbekistan
Group I  |  Netherlands  Jordan  Paraguay  Ghana
Group J  |  Chile  Portugal  Cote d'Ivoire  Morocco
Group K  |  Senegal  Egypt  Spain  England
Group L  |  Tunisia  France  Germany  Algeria


## Knockout stage

32 qualifiers. No draws — every match goes to a winner. The bracket runs
through Round of 32, Round of 16, Quarter-finals, Semi-finals, Third-place
play-off, and the Final.

In [178]:
rounds_played <- 0L

while (!isTRUE(trn$completed)) {
  playable <- matches(trn, "knockout", status = "pending")
  playable <- playable[
    !is.na(playable$participant1) & !is.na(playable$participant2), ,
    drop = FALSE
  ]
  if (nrow(playable) == 0L) break

  for (mid in playable$match_id) {
    trn <- result(trn, "knockout",
      match = mid,
      score = random_score(allow_draw = FALSE)
    )
  }
  rounds_played <- rounds_played + 1L
}

cat("Knockout complete after", rounds_played, "rounds.\n")
stage_status(trn)

Knockout complete after 5 rounds.


     stage   status complete total materialized
1   groups complete       72    72         TRUE
2 knockout complete       32    32         TRUE

In [179]:
cat("\n⚽  FIFA World Cup 2026 Champion:", winner(trn), "\n")


⚽  FIFA World Cup 2026 Champion: Switzerland 


## All 104 results

Every match from the group stage through to the final, with scores and the
winning team.

In [180]:
all_m   <- matches(trn, status = "all")
display <- c("stage_id", "round", "participant1", "score1", "score2",
             "participant2", "winner")
all_m[, intersect(display, names(all_m))]

    stage_id round   participant1 score1 score2   participant2
1     groups     1       Honduras      3      2         Canada
2     groups     1         Poland      2      0        Ukraine
3     groups     2        Ukraine      3      3         Canada
4     groups     2         Poland      2      3       Honduras
5     groups     3         Canada      4      1         Poland
6     groups     3       Honduras      3      4        Ukraine
7     groups     1        Jamaica      2      4         Mexico
8     groups     1         Serbia      3      1          Japan
9     groups     2         Mexico      4      0          Japan
10    groups     2        Jamaica      0      3         Serbia
11    groups     3         Mexico      2      2         Serbia
12    groups     3        Jamaica      4      2          Japan
13    groups     1  United States      0      0         Panama
14    groups     1        Austria      2      4 Korea Republic
15    groups     2  United States      2      0 Korea R